In [1]:
"""
Cell 1：说明 + Jupyter 显示配置

写明 Notebook 目标、文件夹结构（data/raw / data/processed）。

设置 pandas 显示行数、列宽等，只是方便你和导师阅读表格。

brsb_data_generator.ipynb

Purpose:
    Generate semi-synthetic but calibrated data for a Brisbane-based
    lithium-ion battery recycling network:

    1) Demand nodes (Brisbane SA2 / collection points) with multi-grade demand
    2) Facility nodes (recycling / remanufacturing / second-life)
    3) Distance / cost matrix between nodes and facilities (via road network)
    4) Baseline + example disruption scenarios in JSON format

Notes:
    - This notebook will read raw data from:  ./data/raw/
    - And will save processed files to:      ./data/processed/
"""

import pandas as pd
import numpy as np
import osmnx as ox
print(ox.__version__)


# Optional: make pandas tables easier to read in Jupyter
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 20)


2.0.7


In [2]:
"""
Cell 2：导入库 + 路径设置 + osmnx 配置

import pandas, numpy, os, pathlib, osmnx, networkx 等。

定义 BASE_DIR / RAW_DIR / PROCESSED_DIR。

配置 osmnx 的缓存与日志（避免反复下载同一块路网）。
"""


import os
from pathlib import Path

# 地理和绘图相关（后面算距离 / 画图会用到）
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox

# ---- 路径设置 ----
# 当前 Notebook 所在目录（比如 D:\Projects\llm_or_project\Notebook1）
BASE_DIR = Path(".").resolve()

# 原始数据放这里：BASE_DIR\data\raw
RAW_DIR = BASE_DIR / "data" / "raw"

# 生成的数据放这里：BASE_DIR\data\processed
PROCESSED_DIR = BASE_DIR / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("BASE_DIR     :", BASE_DIR)
print("RAW_DIR      :", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)

# osmnx 新版本的配置方式
ox.settings.use_cache = True      # 允许使用本地缓存
ox.settings.log_console = True    # 在控制台输出日志



BASE_DIR     : D:\Projects\llm_or_project\Notebook1
RAW_DIR      : D:\Projects\llm_or_project\Notebook1\data\raw
PROCESSED_DIR: D:\Projects\llm_or_project\Notebook1\data\processed


In [3]:
"""
“Li-ion 占全部便携电池废弃量约 8.3%，来源：B-cycle 2.0 Scenario 5 表 3”

“全国现状收集率约 18%，回收率约 73%，来源：B-cycle webinar 2025 数据”


“Li-ion 质量等级 A:B≈75:25，质量分级定义见 B-cycle Container Protocol”
Cell 3：布里斯班宏观参数

定义 Brisbane 总人口、每人电池数量、报废比例、回收率、Li-ion 占比等参数。

通过这些参数估算 BNE 年回收锂电池总吨数 BNE_RECYCLED_TONNES。

定义 电池等级比例 GRADE_RATIO = {"G1": x, "G2": y, "G3": z}。
"""


# === 3. Macro parameters for Brisbane ===

# 1) 人口：从 ABS Regional Population 取（Greater Brisbane 或 Brisbane LGA）
BNE_POP = 5583833  # 举例：请用你从 ABS 下载的最新人口替换https://dbr.abs.gov.au/region.html?lyr=ste&rgn=3

# 2) 全国 Li-ion 废弃量 & 回收目标
# B-cycle 2.0 Scenario 5 中给出：Li-ion 年废弃量 AfR ≈ 8,800 t
AU_LIION_AFR_T = 8800  

# 同一表中总废电池 AfR ≈106,050 t
AU_ALL_BAT_AFR_T = 106050
LIION_SHARE = AU_LIION_AFR_T / AU_ALL_BAT_AFR_T  # ≈ 8.3%

# 3) 全国收集率：B-cycle webinar 提到收集率约 18%
COLLECTION_RATE_BASE = 0.18

# 4) 假定布里斯班 Li-ion 废弃量 ~ 人口占比 * 全国量
# 你可以先算出“布里斯班占全国人口百分比”，再乘以全国 Li-ion AfR
AU_POP = 27194369   # 用 ABS 全国人口替换
BNE_POP_SHARE = BNE_POP / AU_POP

BNE_LIION_AFR_T = AU_LIION_AFR_T * BNE_POP_SHARE   # Brisbane 年废弃 Li-ion
BNE_LIION_COLLECTED_T = BNE_LIION_AFR_T * COLLECTION_RATE_BASE  # 现状收集量

# 5) 质量等级比例：基于 B-cycle 协议
# B-cycle 对“Li-ion（消费电子/工具/轻出行）”给出 A 75% / B 25%
# 文档中 C 级是“大体积/长滞留”，没有给明确比例
# 为了建模，我们可以假定 C 级占 5%（保守加入高风险流），
# 同时把 A/B 稍微调一下让总和=1：A=70%, B=25%, C=5%
GRADE_RATIO = {
    "A": 0.70,
    "B": 0.25,
    "C": 0.05,
}

print("Brisbane Li-ion AfR (t/yr):", round(BNE_LIION_AFR_T, 1))
print("Brisbane collected Li-ion (t/yr, base):", round(BNE_LIION_COLLECTED_T, 1))
print("Grade ratio:", GRADE_RATIO)


Brisbane Li-ion AfR (t/yr): 1806.9
Brisbane collected Li-ion (t/yr, base): 325.2
Grade ratio: {'A': 0.7, 'B': 0.25, 'C': 0.05}


In [4]:
"""
SA2 边界与人口数据来自 ABS Regional Population & Data by Region
Cell 4：读取 SA2 / 客户节点

从 data/raw/abs_sa2_bne.csv 读取 SA2 级数据（如果没有则自动生成 demo）。

统一列名为 sa2_code, sa2_name, population, lat, lon。

得到 sa2_df：这是后面 customer nodes 的基础。
"""




# ====== Cell 4: Read Brisbane SA2 / customer node data ======

import pandas as pd

sa2_file = RAW_DIR / "abs_sa2_bne.csv"
sa2_df = pd.read_csv(sa2_file)

# 只保留我们需要的列并统一命名
sa2_df = sa2_df.rename(columns={
    "SA2_CODE_2021": "SA2_CODE",
    "SA2_NAME_2021": "SA2_NAME",
    "Population_2023": "POP",
    # 如果经纬度列名不同，这里一起改
})

sa2_df = sa2_df[["SA2_CODE", "SA2_NAME", "POP", "LAT", "LON"]].copy()

print("Number of SA2 in Brisbane:", len(sa2_df))
display(sa2_df.head())


Number of SA2 in Brisbane: 5


,SA2_CODE,SA2_NAME,POP,LAT,LON
0,D01,Demo Area 1,30000,-27.47,153.03
1,D02,Demo Area 2,25000,-27.45,153.01
2,D03,Demo Area 3,20000,-27.49,153.05
3,D04,Demo Area 4,15000,-27.50,153.00
4,D05,Demo Area 5,10000,-27.46,153.04


In [5]:
"""
Cell 5：按人口占比分配总回收量

计算所有 SA2 的总人口，求出每个 SA2 的人口占比 share_pop。

用 share_pop * BNE_RECYCLED_TONNES 得到各 SA2 的 年回收量 recycled_tonnes_total。
"""


# === 5. Allocate Brisbane collected Li-ion to SA2 by population share ===

total_pop = sa2_df["POP"].sum()
sa2_df["pop_share"] = sa2_df["POP"] / total_pop

# 每个 SA2 年收集量（吨）
sa2_df["recycled_tonnes_total"] = sa2_df["pop_share"] * BNE_LIION_COLLECTED_T

print("Check total allocated tonnes:",
      round(sa2_df["recycled_tonnes_total"].sum(), 2))

display(sa2_df[["SA2_CODE", "SA2_NAME", "POP",
                "pop_share", "recycled_tonnes_total"]].head())


Check total allocated tonnes: 325.24


,SA2_CODE,SA2_NAME,POP,pop_share,recycled_tonnes_total
0,D01,Demo Area 1,30000,0.30,97.573047
1,D02,Demo Area 2,25000,0.25,81.310872
2,D03,Demo Area 3,20000,0.20,65.048698
3,D04,Demo Area 4,15000,0.15,48.786523
4,D05,Demo Area 5,10000,0.10,32.524349


In [6]:
"""
Cell 6：按 G1/G2/G3 拆分 + 生成 nodes.csv

用 GRADE_RATIO 把每个 SA2 的总回收量拆成 demand_G1 / demand_G2 / demand_G3。

生成 node_id = C1, C2, …，node_type = "customer"。

导出 data/processed/nodes.csv，字段大致是：

node_id, node_type, sa2_code, sa2_name, lat, lon, population, recycled_tonnes_total, demand_G1, demand_G2, demand_G3
"""


# === 6. Split tonnes by quality grade and export nodes.csv ===

# 按 Cell3 中的 GRADE_RATIO 拆分
sa2_df["grade_A_t"] = sa2_df["recycled_tonnes_total"] * GRADE_RATIO["A"]
sa2_df["grade_B_t"] = sa2_df["recycled_tonnes_total"] * GRADE_RATIO["B"]
sa2_df["grade_C_t"] = sa2_df["recycled_tonnes_total"] * GRADE_RATIO["C"]

# 建立 node_id（后面会在 Gurobi 模型里用）
sa2_df = sa2_df.reset_index(drop=True)
sa2_df["node_id"] = sa2_df.index.map(lambda i: f"N{i+1:03d}")

nodes_cols = [
    "node_id", "SA2_CODE", "SA2_NAME",
    "LAT", "LON",
    "POP", "recycled_tonnes_total",
    "grade_A_t", "grade_B_t", "grade_C_t",
]

nodes_df = sa2_df[nodes_cols].copy()

nodes_path = PROCESSED_DIR / "nodes.csv"
nodes_df.to_csv(nodes_path, index=False, encoding="utf-8-sig")
print("Saved nodes.csv to:", nodes_path)
display(nodes_df.head())



Saved nodes.csv to: D:\Projects\llm_or_project\Notebook1\data\processed\nodes.csv


,node_id,SA2_CODE,SA2_NAME,LAT,LON,POP,recycled_tonnes_total,grade_A_t,grade_B_t,grade_C_t
0,N001,D01,Demo Area 1,-27.47,153.03,30000,97.573047,68.301133,24.393262,4.878652
1,N002,D02,Demo Area 2,-27.45,153.01,25000,81.310872,56.917611,20.327718,4.065544
2,N003,D03,Demo Area 3,-27.49,153.05,20000,65.048698,45.534089,16.262174,3.252435
3,N004,D04,Demo Area 4,-27.50,153.00,15000,48.786523,34.150566,12.196631,2.439326
4,N005,D05,Demo Area 5,-27.46,153.04,10000,32.524349,22.767044,8.131087,1.626217


In [7]:
"""
Cell 7：读取设施 + 生成 facilities.csv

从 data/raw/facilities_raw.csv 读取设施（没有就生成 demo）。

统一列名为 fac_name, fac_type, lat, lon, max_capacity_tonnes, fixed_cost。

加上 fac_id = F1, F2, …，导出 data/processed/facilities.csv。
"""


# Cell 7：读取设施数据，生成 facilities.csv（含等级兼容性和容量）

import pandas as pd
from pathlib import Path

print("RAW_DIR:", RAW_DIR)

fac_raw_file = RAW_DIR / "facilities_raw.csv"
print("fac_raw_file exists?", fac_raw_file.exists())

# 1) 读取原始设施 CSV
fac_raw = pd.read_csv(fac_raw_file)
print("原始列名：", fac_raw.columns.tolist())
display(fac_raw)

# 2) 统一基础列名（不动容量列，单独处理）
fac_raw = fac_raw.rename(columns={
    "FAC_NAME": "name",
    "TYPE": "TYPE",
    "LAT": "LAT",
    "LON": "LON",
    "FIXED_COST": "fixed_cost",
})

# 3) 处理“总容量”这一列：兼容 MAX_CAP / MAX_CAP_TONNES / 已有的 capacity_total_t
cap_candidates = ["capacity_total_t", "MAX_CAP", "MAX_CAP_TONNES"]
cap_col = None
for c in cap_candidates:
    if c in fac_raw.columns:
        cap_col = c
        break

if cap_col is None:
    raise ValueError(
        f"在设施文件中没有找到容量列，请检查列名，当前列为: {fac_raw.columns.tolist()}"
    )

fac_raw["capacity_total_t"] = fac_raw[cap_col].astype(float)
print("使用的容量列是:", cap_col)

# 4) 确保等级相关的标记列存在，并转成 int
for col in ["hazardous_capable", "regional_hub"]:
    if col not in fac_raw.columns:
        fac_raw[col] = 0  # 若列不存在就默认 0，防止报错

fac_raw["hazardous_capable"] = fac_raw["hazardous_capable"].fillna(0).astype(int)
fac_raw["regional_hub"]      = fac_raw["regional_hub"].fillna(0).astype(int)

# 5) 生成是否接受各等级的标记
#    A 级默认所有设施都可以接
fac_raw["accept_A"] = 1
#    B 级：只有具备危险电池处理能力的设施可以接
fac_raw["accept_B"] = fac_raw["hazardous_capable"]
#    C 级：只有区域集中处理中心可以接
fac_raw["accept_C"] = fac_raw["regional_hub"]

# 6) 按 GRADE_RATIO 把总能力拆成 A/B/C 等级能力
#    GRADE_RATIO 已在 Cell 3 中定义，例如：
#    GRADE_RATIO = {"A": 0.3, "B": 0.4, "C": 0.3}
fac_raw["cap_A_t"] = fac_raw["capacity_total_t"] * GRADE_RATIO["A"]
fac_raw["cap_B_t"] = fac_raw["capacity_total_t"] * GRADE_RATIO["B"] * fac_raw["accept_B"]
fac_raw["cap_C_t"] = fac_raw["capacity_total_t"] * GRADE_RATIO["C"] * fac_raw["accept_C"]

# 7) 生成 facility_id，并选取需要输出的列
fac_raw = fac_raw.reset_index(drop=True)
fac_raw["facility_id"] = fac_raw.index.map(lambda i: f"F{i+1:02d}")

fac_cols = [
    "facility_id", "name", "TYPE",
    "LAT", "LON",
    "accept_A", "accept_B", "accept_C",
    "capacity_total_t", "cap_A_t", "cap_B_t", "cap_C_t",
    "fixed_cost",
]

facilities_df = fac_raw[fac_cols].copy()

# 8) 保存到 processed 目录
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
facilities_path = PROCESSED_DIR / "facilities.csv"
facilities_df.to_csv(facilities_path, index=False, encoding="utf-8-sig")

print("✅ Saved facilities.csv to:", facilities_path)
display(facilities_df)



RAW_DIR: D:\Projects\llm_or_project\Notebook1\data\raw
fac_raw_file exists? True
原始列名： ['FAC_NAME', 'TYPE', 'LAT', 'LON', 'MAX_CAP_TONNES', 'FIXED_COST', 'hazardous_capable', 'regional_hub']


,FAC_NAME,TYPE,LAT,LON,MAX_CAP_TONNES,FIXED_COST,hazardous_capable,regional_hub
0,Demo Recycling Center 1,recycling,-27.48,153.02,50,200,1,1
1,Demo Remanufacturing Plant 1,remanufacturing,-27.44,153.04,30,250,1,0
2,Demo Second-life Facility 1,second_life,-27.50,153.00,40,180,0,0


使用的容量列是: MAX_CAP_TONNES
✅ Saved facilities.csv to: D:\Projects\llm_or_project\Notebook1\data\processed\facilities.csv


,facility_id,name,TYPE,LAT,LON,accept_A,accept_B,accept_C,capacity_total_t,cap_A_t,cap_B_t,cap_C_t,fixed_cost
0,F01,Demo Recycling Center 1,recycling,-27.48,153.02,1,1,1,50.0,35.0,12.5,2.5,200
1,F02,Demo Remanufacturing Plant 1,remanufacturing,-27.44,153.04,1,1,0,30.0,21.0,7.5,0.0,250
2,F03,Demo Second-life Facility 1,second_life,-27.50,153.00,1,0,0,40.0,28.0,0.0,0.0,180


In [8]:
# =========================
# Cell 8: Brisbane road network (robust + reproducible)
# - Ensure nodes_df / facilities_df exist (auto-load from processed if needed)
# - Compute bbox from data
# - Try download OSM road network (drive)
# - Save graphml + PNG
# - If download fails: set ROAD_GRAPH=None (Cell 9 will fallback to haversine)
# =========================

from pathlib import Path
import pandas as pd
import osmnx as ox
import matplotlib.pyplot as plt

print("==== Cell 8: Download Brisbane road network (with graceful fallback) ====")

# ------------------------------------------------------------
# 0) Preconditions: ensure PROCESSED_DIR exists
# ------------------------------------------------------------
if "PROCESSED_DIR" not in globals():
    raise RuntimeError("PROCESSED_DIR is not defined. Please run Cell 2 first.")

PROCESSED_DIR = Path(PROCESSED_DIR)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# 1) Ensure nodes_df / facilities_df exist (robust after Restart Kernel)
# ------------------------------------------------------------
nodes_path = PROCESSED_DIR / "nodes.csv"
fac_path   = PROCESSED_DIR / "facilities.csv"

if "nodes_df" not in globals():
    if nodes_path.exists():
        nodes_df = pd.read_csv(nodes_path)
        print("[OK] Loaded nodes_df from:", nodes_path)
    else:
        raise FileNotFoundError(f"[ERROR] nodes.csv not found at {nodes_path}. Please run Cell 4-6 first.")

if "facilities_df" not in globals():
    if fac_path.exists():
        facilities_df = pd.read_csv(fac_path)
        print("[OK] Loaded facilities_df from:", fac_path)
    else:
        raise FileNotFoundError(f"[ERROR] facilities.csv not found at {fac_path}. Please run Cell 7 first.")

# ------------------------------------------------------------
# 2) Column checks (your files use LAT/LON in uppercase)
# ------------------------------------------------------------
required_nodes_cols = {"LAT", "LON"}
required_fac_cols   = {"LAT", "LON"}

missing_nodes = required_nodes_cols - set(nodes_df.columns)
missing_fac   = required_fac_cols - set(facilities_df.columns)

if missing_nodes:
    raise KeyError(f"[ERROR] nodes_df missing columns: {missing_nodes}. Found: {list(nodes_df.columns)}")
if missing_fac:
    raise KeyError(f"[ERROR] facilities_df missing columns: {missing_fac}. Found: {list(facilities_df.columns)}")

# ------------------------------------------------------------
# 3) Compute bbox from nodes+facilities (add margin)
# ------------------------------------------------------------
lat_min = float(min(nodes_df["LAT"].min(), facilities_df["LAT"].min()) - 0.05)
lat_max = float(max(nodes_df["LAT"].max(), facilities_df["LAT"].max()) + 0.05)
lon_min = float(min(nodes_df["LON"].min(), facilities_df["LON"].min()) - 0.05)
lon_max = float(max(nodes_df["LON"].max(), facilities_df["LON"].max()) + 0.05)

print(f"BBox lat[{lat_min:.3f}, {lat_max:.3f}], lon[{lon_min:.3f}, {lon_max:.3f}]")

# OSMnx v2.x expects bbox tuple: (west, south, east, north)
bbox = (lon_min, lat_min, lon_max, lat_max)

# ------------------------------------------------------------
# 4) Download road network (drive). If fail, fallback to None
# ------------------------------------------------------------
ROAD_GRAPH = None

try:
    # Optional: cache and logging (safe for v2.x)
    try:
        ox.settings.use_cache = True
        ox.settings.log_console = True
    except Exception:
        pass

    print("[INFO] Trying to download road network from OSM/Overpass (drive)...")
    G = ox.graph_from_bbox(bbox=bbox, network_type="drive")
    print("[OK] Downloaded raw graph: nodes =", len(G.nodes), ", edges =", len(G.edges))

    # Project graph for metric distances
    G_proj = ox.project_graph(G)
    print("[OK] Projected graph: nodes =", len(G_proj.nodes), ", edges =", len(G_proj.edges))

    # Save graphml (reproducible)
    graphml_path = PROCESSED_DIR / "brisbane_road_network.graphml"
    ox.save_graphml(G_proj, filepath=graphml_path)
    print("[OK] Saved graphml to:", graphml_path)

    # Save PNG figure for paper/report
    fig, ax = ox.plot_graph(
        G_proj,
        node_size=0,
        edge_linewidth=0.4,
        bgcolor="white",
        show=False,
        close=False
    )
    fig_path = PROCESSED_DIR / "brisbane_road_network.png"
    fig.savefig(fig_path, dpi=200, bbox_inches="tight")
    plt.close(fig)
    print("[OK] Saved road network figure to:", fig_path)

    ROAD_GRAPH = G_proj

except Exception as e:
    print("\n[WARNING] Failed to download road network via OSMnx.")
    print("Reason:", repr(e))
    print("=> We will fallback to straight-line (haversine) distances in Cell 9.")
    ROAD_GRAPH = None

print("[DONE] Cell 8 finished. ROAD_GRAPH is", "READY" if ROAD_GRAPH is not None else "None (fallback mode)")


==== Cell 8: Download Brisbane road network (with graceful fallback) ====
BBox lat[-27.550, -27.390], lon[152.950, 153.100]
[INFO] Trying to download road network from OSM/Overpass (drive)...
[OK] Downloaded raw graph: nodes = 18298 , edges = 43343
[OK] Projected graph: nodes = 18298 , edges = 43343
[OK] Saved graphml to: D:\Projects\llm_or_project\Notebook1\data\processed\brisbane_road_network.graphml
[OK] Saved road network figure to: D:\Projects\llm_or_project\Notebook1\data\processed\brisbane_road_network.png
[DONE] Cell 8 finished. ROAD_GRAPH is READY


In [9]:
"""
Cell 9：最近路网节点 + 最短路径距离矩阵 → dist.csv

用 ox.distance.nearest_nodes(G, X=lon, Y=lat) 为每个 customer & facility 找最近路网节点。

在 G_proj 上用 nx.shortest_path_length(weight="length") 算每一对 (node, facility) 的最短路程（米 → 公里）。

生成 dist_df 并导出 data/processed/dist.csv：

from_node, to_facility, distance_km
"""



# ==== Cell 9: Build distance matrix between nodes and facilities ====

import math
import networkx as nx
import pandas as pd

# 先读入我们之前生成的 nodes.csv 和 facilities.csv
nodes_path = PROCESSED_DIR / "nodes.csv"
fac_path   = PROCESSED_DIR / "facilities.csv"

nodes_df = pd.read_csv(nodes_path)
fac_df   = pd.read_csv(fac_path)

print("Number of customer nodes:", len(nodes_df))
print("Number of facilities:", len(fac_df))

# 为了兼容大小写，自动识别 lat / LAT, lon / LON
lat_col_nodes = "LAT" if "LAT" in nodes_df.columns else "lat"
lon_col_nodes = "LON" if "LON" in nodes_df.columns else "lon"

lat_col_fac = "LAT" if "LAT" in fac_df.columns else "lat"
lon_col_fac = "LON" if "LON" in fac_df.columns else "lon"

# --------- 情况 A：有路网，用最短路径距离 ---------
if ROAD_GRAPH is None:
    print("\nUsing road network shortest-path distances.")

    G_proj = ROAD_GRAPH

    # 1) 把经纬度对应到路网图上的最近节点
    import osmnx as ox
    import numpy as np

    nodes_points = nodes_df[[lon_col_nodes, lat_col_nodes]].to_numpy()
    fac_points   = fac_df[[lon_col_fac,   lat_col_fac]].to_numpy()

    # 最近图节点 id
    nearest_nodes_customers = ox.distance.nearest_nodes(
        G_proj, X=nodes_points[:, 0], Y=nodes_points[:, 1]
    )
    nearest_nodes_facilities = ox.distance.nearest_nodes(
        G_proj, X=fac_points[:, 0], Y=fac_points[:, 1]
    )

    # 2) 计算所有 “客户节点 – 设施” 之间的最短路径长度
    rows = []
    for i, node_row in nodes_df.iterrows():
        node_id = node_row["node_id"]
        gn = nearest_nodes_customers[i]

        for j, fac_row in fac_df.iterrows():
            fac_id = fac_row["facility_id"]
            gf = nearest_nodes_facilities[j]

            try:
                dist_m = nx.shortest_path_length(G_proj, gn, gf, weight="length")
            except nx.NetworkXNoPath:
                # 没有路相连，就用一个很大的数表示不可达
                dist_m = 1e9

            rows.append({
                "node_id": node_id,
                "facility_id": fac_id,
                "distance_m": dist_m
            })

    dist_df = pd.DataFrame(rows)

# --------- 情况 B：没有路网，用直线距离 (haversine) 近似 ---------
else:
    print("\nNo road network available. Using straight-line (haversine) distances as approximation.")

    def haversine(lat1, lon1, lat2, lon2):
        # 半径（地球），单位：米
        R = 6371000.0
        phi1, phi2 = math.radians(lat1), math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        a = math.sin(dphi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    rows = []
    for _, node_row in nodes_df.iterrows():
        node_id = node_row["node_id"]
        lat_n = node_row[lat_col_nodes]
        lon_n = node_row[lon_col_nodes]

        for _, fac_row in fac_df.iterrows():
            fac_id = fac_row["facility_id"]
            lat_f = fac_row[lat_col_fac]
            lon_f = fac_row[lon_col_fac]

            d = haversine(lat_n, lon_n, lat_f, lon_f)

            # 如果你想模仿“路程比直线略长”，可以乘一个系数，比如 1.2：
            road_like_d = d * 1.2

            rows.append({
                "node_id": node_id,
                "facility_id": fac_id,
                "distance_m": road_like_d
            })

    dist_df = pd.DataFrame(rows)

# 3) 透视成矩阵形式，并导出 dist.csv
dist_matrix = dist_df.pivot(index="node_id", columns="facility_id", values="distance_m")
dist_path = PROCESSED_DIR / "dist.csv"
dist_matrix.to_csv(dist_path)
print("\nSaved dist.csv to:", dist_path)
display(dist_matrix.head())



Number of customer nodes: 5
Number of facilities: 3

No road network available. Using straight-line (haversine) distances as approximation.

Saved dist.csv to: D:\Projects\llm_or_project\Notebook1\data\processed\dist.csv


facility_id,F01,F02,F03
node_id,,,
N001,1783.800076,4174.462650,5351.186227
N002,4174.432191,3794.820600,6775.913505
N003,3793.614030,6775.932275,6066.694582
N004,3567.386172,9301.738135,0.000000
N005,3567.671441,2668.678239,7135.057557


In [15]:
"""
Cell 10：构造情景集 → scenarios.json

用 facilities_df 中第一个设施 + population 最大的 SA2 作为示例：

S0：基准场景（不调整容量/需求）；

S1：某设施容量降为 40%；

S2：某客户区域需求 ×1.5；

S3：设施 70% + 主要客户需求 1.3 倍（组合场景）。

每个场景是一个 dict：包含 facility_capacity_factors 和 customer_demand_factors 等。

列表写入 data/processed/scenarios.json。
"""

# ==== Cell 10: 生成 scenarios.json（baseline + 示例扰动场景）====

import json
import pandas as pd

# 1) 读取已经生成好的 nodes / facilities
nodes_path = PROCESSED_DIR / "nodes.csv"
fac_path   = PROCESSED_DIR / "facilities.csv"

nodes_df      = pd.read_csv(nodes_path)
facilities_df = pd.read_csv(fac_path)

print("Nodes:", len(nodes_df), "Facilities:", len(facilities_df))
print("Facility columns:", list(facilities_df.columns))

# 2) 兼容不同列名：优先使用 facility_id / name
fac_df = facilities_df.copy()

id_col = "facility_id" if "facility_id" in fac_df.columns else "FAC_ID"
name_col = "name" if "name" in fac_df.columns else "FAC_NAME"

# 确保是字符串，方便后面写入 JSON
fac_df[id_col] = fac_df[id_col].astype(str)

# 取前两个设施，作为示例场景里的“R1 / R3”替身
fac1_id   = fac_df.iloc[0][id_col]
fac1_name = fac_df.iloc[0][name_col]

if len(fac_df) > 1:
    fac2_id   = fac_df.iloc[1][id_col]
    fac2_name = fac_df.iloc[1][name_col]
else:
    fac2_id, fac2_name = fac1_id, fac1_name  # 只有一个设施时就重复用
    
if len(fac_df) > 2:
    fac3_id   = fac_df.iloc[2][id_col]
    fac3_name = fac_df.iloc[2][name_col]
else:
    fac3_id, fac3_name = fac2_id, fac2_name

# 3) 定义基准场景 + 两个示例扰动场景
scenarios = {
    "baseline": {
        "summary_zh": "基准场景：所有回收设施正常运行。",
        "affected_center": None,
        "capacity_factor": 1.0,
        "duration_days": 0,
        "note": "用于作为成本与服务水平对比的参考基线。"
    },

    f"{fac1_id}_40pct_5days": {
        "summary_zh": f"场景1：设施 {fac1_name}（{fac1_id}）发生故障，未来5天处理能力降为40%。",
        "affected_center": fac1_id,
        "capacity_factor": 0.4,
        "duration_days": 5
    },

    f"{fac2_id}_80pct_10days": {
        "summary_zh": f"场景2：设施 {fac2_name}（{fac2_id}）计划维护，未来10天处理能力降为80%。",
        "affected_center": fac2_id,
        "capacity_factor": 0.8,
        "duration_days": 10
    },
    f"{fac3_id}_60pct_7days": {
    "summary_zh": f"场景3：设施 {fac3_name}（{fac3_id}）位于偏远区域，未来7天处理能力降为60%。",
    "affected_center": fac3_id,
    "capacity_factor": 0.6,
    "duration_days": 7
}
}

# 4) 保存为 scenarios.json
scenarios_path = PROCESSED_DIR / "scenarios.json"
with open(scenarios_path, "w", encoding="utf-8") as f:
    json.dump(scenarios, f, indent=2, ensure_ascii=False)

print("Saved scenarios.json to:", scenarios_path)



Nodes: 5 Facilities: 3
Facility columns: ['facility_id', 'name', 'TYPE', 'LAT', 'LON', 'accept_A', 'accept_B', 'accept_C', 'capacity_total_t', 'cap_A_t', 'cap_B_t', 'cap_C_t', 'fixed_cost']
Saved scenarios.json to: D:\Projects\llm_or_project\Notebook1\data\processed\scenarios.json
